<a href="https://colab.research.google.com/github/FredLongo/MLDL/blob/main/CNN_Model_to_Train_Touching_R2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Fred Longo, Modeling after notebook:

https://www.kaggle.com/code/shuvojitdas/emotion-detection-using-cnn/notebook

### This is where the Program Starts

In [1]:

#Import Librarys

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [2]:

from datetime import datetime
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers import MaxPooling2D
from keras.models import Model, Sequential
#from tensorflow.python.keras.optimizer_v2.adam import Adam
#from keras.optimizers import adam_v2
from tensorflow.keras.optimizers import RMSprop
#from keras.optimizers import rmsprop
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam

In [3]:
# Setting

picture_size = 224
folder_path       = "drive/MyDrive/Databases/Dataset_Project_1/"
folder_path_train = "drive/MyDrive/Databases/Dataset_Project_1/train"
no_of_classes = 3
epochs = 20





### Set Up Data groups for Training Testing and Validation.

In [4]:
batch_size  = 10

'''
# Define your ImageDataGenerator with a validation split
datagen = ImageDataGenerator(validation_split=0.2) # for example, 20% for validation

# Setup the training generator
train_generator = datagen.flow_from_directory(
    directory=folder_path_train, # The directory where your data is located
    target_size=(picture_size, picture_size),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode='categorical',
    subset='training', # Set as training data
    shuffle=True)


# Setup the validation generator
validation_generator = datagen.flow_from_directory(
    directory=folder_path_train, # The directory where your data is located
    target_size=(picture_size, picture_size),
    color_mode="grayscale",
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation', # Set as validation data
    shuffle=True)


'''

# Original


datagen_train  = ImageDataGenerator()
datagen_val = ImageDataGenerator()

train_folder_path = folder_path + "small_v"
validate_folder_path = folder_path + "small_t"

train_set = datagen_train.flow_from_directory(train_folder_path,
                                              target_size = (picture_size,picture_size),
                                              color_mode = "grayscale",
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              shuffle=True)


test_set = datagen_val.flow_from_directory(validate_folder_path,
                                              target_size = (picture_size,picture_size),
                                              color_mode = "grayscale",
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              shuffle=False)



Found 960 images belonging to 3 classes.
Found 960 images belonging to 3 classes.


### Build Model

In [5]:

model = Sequential()

#1st CNN layer
#model.add(Conv2D(64,(3,3),padding = 'same',input_shape = (48,48,1)))
model.add(Conv2D(64,(3,3),padding = 'same',input_shape = (224,224,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

#2nd CNN layer
model.add(Conv2D(128,(5,5),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.25))

#3rd CNN layer
model.add(Conv2D(512,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.25))

#4th CNN layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

#Fully connected 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))


# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(no_of_classes, activation='softmax'))



opt = Adam(learning_rate=0.0001)

In [6]:
model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      640       
                                                                 
 batch_normalization (Batch  (None, 224, 224, 64)      256       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 224, 224, 64)      0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 112, 112, 64)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 128)     2

In [8]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint("./model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

early_stopping = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_learningrate = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks_list = [early_stopping,checkpoint,reduce_learningrate]

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
print("Start Time:",(datetime.now()).strftime("%H:%M:%S"))

steps_per_epoch = train_set.n//train_set.batch_size
if steps_per_epoch == 0:
    steps_per_epoch = 1
print(steps_per_epoch)
# Original

history = model.fit_generator(generator=train_set,
#                                steps_per_epoch=steps_per_epoch,
                                epochs=epochs,
                                validation_data = test_set,
                                validation_steps = test_set.n//test_set.batch_size,
                                callbacks=callbacks_list
                                )



print("End Time:",(datetime.now()).strftime("%H:%M:%S"))



Start Time: 06:16:09
96


<ipython-input-9-e3e871e213d2>:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator=train_set,


Epoch 1/20
96/96 [==============================] - ETA: 0s - loss: 1.4254 - accuracy: 0.3583 
Epoch 1: val_accuracy improved from -inf to 0.32396, saving model to ./model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


96/96 [==============================] - 1436s 15s/step - loss: 1.4254 - accuracy: 0.3583 - val_loss: 3.5439 - val_accuracy: 0.3240 - lr: 0.0010
Epoch 2/20
96/96 [==============================] - ETA: 0s - loss: 1.2021 - accuracy: 0.4458 
Epoch 2: val_accuracy improved from 0.32396 to 0.33229, saving model to ./model.h5
96/96 [==============================] - 1309s 14s/step - loss: 1.2021 - accuracy: 0.4458 - val_loss: 2.7566 - val_accuracy: 0.3323 - lr: 0.0010
Epoch 3/20
96/96 [==============================] - ETA: 0s - loss: 1.1007 - accuracy: 0.4948 
Epoch 3: val_accuracy improved from 0.33229 to 0.45937, saving model to ./model.h5
96/96 [==============================] - 1450s 15s/step - loss: 1.1007 - accuracy: 0.4948 - val_loss: 1.1327 - val_accuracy: 0.4594 - lr: 0.0010
Epoch 4/20
96/96 [==============================] - ETA: 0s - loss: 1.0173 - accuracy: 0.5385 
Epoch 4: val_accuracy improved from 0.45937 to 0.50104, saving model to ./model.h5
96/96 [========================

In [2]:
# @title Default title text
'''
print("Start Time:",(datetime.now()).strftime("%H:%M:%S"))

#Modified

history = model.fit(generator=train_generator,
                                steps_per_epoch=train_generator.n//train_generator.batch_size,
                                epochs=epochs,
                                steps_per_epoch = epochs,
                                validation_data = validation_generator,
                                validation_steps = validation_generator.n//validation_generator.batch_size,
                                callbacks=callbacks_list
                                )




print("End Time:",(datetime.now()).strftime("%H:%M:%S"))
'''


'\nprint("Start Time:",(datetime.now()).strftime("%H:%M:%S"))\n\n#Modified\n\nhistory = model.fit(generator=train_generator,\n                                steps_per_epoch=train_generator.n//train_generator.batch_size,\n                                epochs=epochs,\n                                steps_per_epoch = epochs,\n                                validation_data = validation_generator,\n                                validation_steps = validation_generator.n//validation_generator.batch_size,\n                                callbacks=callbacks_list\n                                )\n\n\n\n\nprint("End Time:",(datetime.now()).strftime("%H:%M:%S"))\n'

In [ ]:
plt.style.use('dark_background')

plt.figure(figsize=(20,10))
plt.subplot(1, 2, 1)
plt.suptitle('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.show()